Tiering of BloodHound data

Import libs

In [13]:
from neo4j import GraphDatabase
import pathlib

Create connection

In [16]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
                
        return response
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="bloodhound")
relpath = pathlib.Path().absolute().as_uri()

Tier the data

In [54]:
query = '''
CALL apoc.load.json('%s/../jsondata/bloodhound/tiering-bloodhound.json') YIELD value

// Users
UNWIND value.users as u
MERGE (tier:Tier {level:u.tier})
WITH value, tier, u
MATCH (user:User) WHERE user.objectid ENDS WITH u.RID
MERGE (user)-[tierrel:InTier]->(tier)

// Groups
WITH value
UNWIND value.groups as g
MERGE (tier:Tier {level:g.tier})
WITH value, tier, g
MATCH (group:Group) WHERE group.objectid ENDS WITH g.RID
MATCH (principal)-[:MemberOf*1..]->(group) WHERE (principal:User) OR (principal:Computer)
MERGE (principal)-[tierrel:InTier]->(tier)

// OUs
WITH value
UNWIND value.ous as o
MERGE (tier:Tier {level:o.tier})
WITH value, tier, o
MATCH (principal) WHERE (principal.distinguishedname ENDS WITH o.path) AND ((principal:User) OR (principal:Computer))
MERGE (principal)-[tierrel:InTier]->(tier)
''' % (relpath)
out = conn.query(query)

# Delete the Tier 1 and/or 2 relations for principles in multiple tiers
query = '''
MATCH (principal) WHERE size((principal)-[:InTier]->()) > 1
MATCH (principal)-[:InTier]->(tier)
WITH principal, min(tier.level) AS lowestTierLevel
MATCH (principal)-[r:InTier]->(tier) WHERE tier.level > lowestTierLevel
DELETE r
'''
out = conn.query(query)

Get names of objects in a given tier

In [ ]:
query = '''
MATCH (principal)-[:InTier]->(:Tier {level:"0"}) RETURN principal.name LIMIT 25
'''
out = conn.query(query)

Get number of principals in a given tier

In [56]:
query = '''
MATCH (principal)-[:InTier]->(:Tier {level:"2"}) RETURN COUNT(principal)
'''
out = conn.query(query)

Clean graph from Tier nodes and relations

In [53]:
query = '''
MATCH (n:Tier) DETACH DELETE n
'''
out = conn.query(query)